Vorbereitung der Umgebung
Benötigte Packete
-nympy
-matplotlib
-gstools

-jupyter-notebook Umgebung (z.B. jupyterlab)

- jupyterlab mit "jupyter-lab" in Konsole ausführen



In [ ]:
#Importiere benötgte Pakete
import numpy as np
from matplotlib import pyplot as plt
import gstools as gs

In [ ]:
#Importiere alle für Hand-Ons benötigten Funktionen
from ml_kriging import calculateKrigingModel, plotKrigModel, plotCovarianceModel, getSEmapOfKrigModel, getBestCovarianceModel, plotFunction2D
from ml_kriging import getPointWithMaxValue, addOnePoint, getMSE, analyseFunction 
from ml_kriging import addRandomPoints, addEdgePoints, getRandomPoints, getGridPoints, getCenteredGridPoints, getCenteredRandomGridPoints, getCenteredRandomGridCombinedPoints, getCenteredRandomGridCombinedPointsWithEdges, getInitPoints
from ml_kriging import f1, f2, f3, f_herbie_1, f_herbie_2, f_herbie_3, f_herbie_4, f_herbie_complex, f_rosenbrock, f_zum_versagen_von_kriging, f_shubert_1_not_for_hand_ons, f_shubert_2_not_for_hand_ons, f_parabolid, f_hyperbolic_parabolid, f_eggholder, f_six_hump_camel, f_three_hump_camel, f_branin  


#####################################################
##############  HAND-ONs: Einleitung   ##############
#####################################################

# Erklärung der vorhanden Funktionen

distribution_type
"RandomPoints" (bzw. "else-case),           #zufällige verteilung
"GridPoints"                                #im gitter angeordnet mit randpunkten
"CenteredGridPoints"                        #im gitter angeordnet, ohne randpunkte
"CenteredRandomGridPoints"                  #in gitter aufgeteilt und im quadranten eine zufallszahl gewählt
"CenteredRandomGridCombinedPoints"          #kombination von random-grid und random punkten
"CenteredRandomGridCombinedPointsWithEdges" #kombination auf random-grid und random punkten +4 eckpunkte
x_p,y_p = getInitPoints(distribution_type, n_datapoints, min_x,max_x, min_y,max_y)

In dieser Funktion werden die einzelnen Funktionen aufgerufen: addRandomPoints, addEdgePoints, getRandomPoints, getGridPoints, getCenteredGridPoints, getCenteredRandomGridPoints, getCenteredRandomGridCombinedPoints, getCenteredRandomGridCombinedPointsWithEdges
- - - - - - - - -     
plotFunction2D(f, ax) #plotted die in handons verwendete funktionen
- - - - - - - - -   
kriging_type
"simple"
"ordinary"
"universal_linear"
"universal_quadratic"
calculateKrigingModel(kriging_type, model_in, x_in, y_in, z_in) 
- - - - - - - - -   
plotKrigModel #plotet das kriging-model
- - - - - - - - -   
plotCovarianceModel #plotet das kovarianzmodel
- - - - - - - - -   
getSEmapOfKrigModel #berechnet quadratische abweichung von wahrer funktion
- - - - - - - - -   

        "Gaussian": gs.Gaussian,
        "Exponential": gs.Exponential,
        "Matern": gs.Matern,
        "Stable": gs.Stable,
        "Rational": gs.Rational,
        "Circular": gs.Circular,
        "Spherical": gs.Spherical,
        "SuperSpherical": gs.SuperSpherical
getBestCovarianceModel(x_p, y_p, z_p, textoutput = False, models) #fittet die übergebenen Kovarianzmodelle und gibt das model mit dem besten pseudo-r2 wert zurück
- - - - - - - - -   
getPointWithMaxValue #gibt den punkt x,y mit dem größten erwarteten Krigingmodelfehler zurück
- - - - - - - - -   
addOnePoint #fügt einen punkt zum bestehen array
- - - - - - - - -   
getMSE #berechnet mean squared error von einem array
- - - - - - - - - - - -
Mit dieser Funktion kann der MSE in Abhängigkeit von der Anzahl der Startpunkte untersuchen werden.
Man kann in diesem notebook definierte Funktion damit untersuchen
Die Logik führt zum Einen den Fit mit allen Punkten durch, zum Anderen wird zusätzlich mit Aufsplittung in Startpunkte + größter SE das Model laufend verbessert
analyseFunction(f1, n_points_to_fit, n_runs, init_fraction, fig, ax1, ax2, ax3, ax4)
- - - - - - - - - - - -
f1, f2, f3, .... sind die definierten Funktionen als Klassen, die in Handons verwendet werden, mit
 .getValue(x,y) zur Rückgab des funktionwertes
 .getRange() zur Rückgabe des Wertebereichs

Liste der Funktionen:
- f1
- f2
- f3
- f_herbie_1
- f_herbie_2
- f_herbie_3
- f_herbie_4
- f_rosenbrock
- f_zum_versagen_von_kriging
- f_parabolid
- f_hyperbolic_parabolid
- f_eggholder
- f_six_hump_camel
- f_three_hump_camel
- f_branin 

Funktionsklasse
#Erstelle deine eigne funktion, oder verwende eine der bereits definierten
    class f_deine_eigene_funktion:
        def __init__(self):
            #definiere Wertebereich der Funktion
            self.min_x = -1
            self.max_x = 1
            self.min_y = -1
            self.max_y = 1

        def getRange(self):
            return self.min_x, self.max_x, self.min_y, self.max_y

        def getValue(self, x, y):
            #definiere funktion
            #z.b. return x + y (geneigte platte)
            #z.b. return x**2 + y**2 Paraboloid
            return x**2-y**2
            
Weitere Funktionen: https://www.sfu.ca/~ssurjano/optimization.html

# ##################################################################
# # HANDS-ON: Einleitung (Theorie -> Praxisbeispiel)  ##############
# ##################################################################

In [ ]:
#wähle die funktion aus
f_current = f_hyperbolic_parabolid()

#extrahiere wertebereich
min_x, max_x, min_y, max_y = f_current.getRange()

fig, ax = plt.subplots()
plotFunction2D(f_current, ax)

# Erklärung Stützpunkte

In [ ]:
fig, ax = plt.subplots()
ax.title.set_text('Auswahl Stützpunkte')

#definiere parameter
n_datapoints = 64
#distribution_type
# / "RandomPoints" (bzw. "else-case),
# / "GridPoints"
# / "CenteredGridPoints"
# / "CenteredRandomGridPoints"
# / "CenteredRandomGridCombinedPoints"
# / "CenteredRandomGridCombinedPointsWithEdges"
x_p,y_p = getInitPoints("GridPoints", n_datapoints, min_x,max_x, min_y,max_y)

plt.xlim(min_x, max_x)
plt.ylim(min_y, max_y)
ax.scatter(x_p,y_p, color = "red")


# Erklärung Variogram

In [ ]:
#definiere parameter
n_known_datapoints = 50
n_points_to_plot = 100

#distribution_type
# / "RandomPoints" (bzw. "else-case),
# / "GridPoints"
# / "CenteredGridPoints"
# / "CenteredRandomGridPoints"
# / "CenteredRandomGridCombinedPoints"
# / "CenteredRandomGridCombinedPointsWithEdges"
x_p,y_p = getInitPoints("CenteredRandomGridCombinedPoints", n_known_datapoints, min_x,max_x, min_y,max_y)

#berechne funktionswerte
z_p = f_current.getValue(x_p,y_p)

fig, ax = plt.subplots()

bin_center, gamma = gs.vario_estimate((x_p,y_p), z_p)
bin_center = bin_center[gamma > 0]
gamma = gamma[gamma > 0]

models = {
    "Gaussian": gs.Gaussian,
    #"Exponential": gs.Exponential,
    "Matern": gs.Matern,
    "Stable": gs.Stable,
    #"Rational": gs.Rational,
    #"Circular": gs.Circular,
    #"Spherical": gs.Spherical,
    #"SuperSpherical": gs.SuperSpherical,
    #"JBessel": gs.JBessel, #keine Varianzberechnung möglich !!
}


# plot the estimated variogram
ax.scatter(bin_center, gamma, color="k", label="data")


# fit all models to the estimated variogram
scores = {}
max_distance_variogram = 0.75*max(max_x-min_x, max_y-min_y)
for model in models:
    fit_model = models[model](dim=2)
    para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True, nugget=False)
    fit_model.plot(x_max=max_distance_variogram, ax=ax)
    scores[model] = r2

ranking = sorted(scores.items(), key=lambda item: item[1], reverse=True)
print("RANKING by Pseudo-r2 score")
best_model = {}
best_score = -1
for i, (model, score) in enumerate(ranking, 1):
    if score > best_score:
        best_score = score
        best_model = models[model](dim=2)
    print(f"{i:>6}. {model:>15}: {score:.5}")




# Erklärung Kriging-Model(s)

In [ ]:
#bereite punkte zum plotten 
x,y = np.meshgrid(np.linspace(min_x,max_x,n_points_to_plot),np.linspace(min_y,max_y,n_points_to_plot))
z = f_current.getValue(x, y)
z_arr1d = np.reshape(z, n_points_to_plot**2)

#berechne kriging-model
#"simple"
#"ordinary"
#"universal_linear"
#"universal_quadratic"
krig = calculateKrigingModel("universal_linear", best_model, x_p,y_p, z_p)

fig, (ax1, ax2) = plt.subplots(1,2)
plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
fig.set_size_inches(15,5)


-----------------------------------------------------------------------------
# -----------------------------------------------------------
# HANDS-ON: Variogrammen erstellen und verstehen 
# -----------------------------------------------------------
-----------------------------------------------------------------------------

# HANDS-ON: Varianzaufgabe 1 

In [ ]:
#Erstelle deine eigene funktion, oder verwende eine der bereits definierten
class f_deine_eigene_funktion:
    def __init__(self):
        #definiere Wertebereich der Funktion
        self.min_x = 0
        self.max_x = 1
        self.min_y = 0
        self.max_y = 1

    def getRange(self):
        return self.min_x, self.max_x, self.min_y, self.max_y

    def getValue(self, x, y):
        #definiere funktion
        return x + y

#wähle die funktion aus
f_bekannte_funktion = f_deine_eigene_funktion()    #wenn du die von dir definierte funktion verwenden möchtest
#f_bekannte_funktion = f2()                         #wenn du z.b. die hier implementierte f2

#funktion zum anschauen der funktion
fig, ax = plt.subplots()
plotFunction2D(f_bekannte_funktion, ax)

In [ ]:
#extrahiere wertebereich
min_x, max_x, min_y, max_y = f_bekannte_funktion.getRange()


#fitte variogram (bei dieser Aufgabe gerne mehr Punkte verwenden, es geht ums Verständnis der Variogramme
#startpunkte
n_known_datapoints = 100

#distribution_type
# / "RandomPoints" (bzw. "else-case),
# / "GridPoints"
# / "CenteredGridPoints"
# / "CenteredRandomGridPoints"
# / "CenteredRandomGridCombinedPoints"
# / "CenteredRandomGridCombinedPointsWithEdges"
x_p,y_p = getInitPoints("CenteredRandomGridCombinedPoints", n_known_datapoints, min_x,max_x, min_y,max_y)
z_p = f_bekannte_funktion.getValue(x_p,y_p)


#berechne experimentelles variogram

bin_center, gamma = gs.vario_estimate((x_p, y_p), z_p)
#enferne fehlende distanzen
bin_center = bin_center[gamma > 0]
gamma = gamma[gamma > 0]

#schaue das ergebnis an

fig, ax = plt.subplots()
# plot the estimated variogram
ax.scatter(bin_center, gamma, color="k", label="data")

Welche fit-funktion würde sich hier anbieten ? (den Verlauf in etwa, ich weiß selber nicht wie die alle aussehen ;) )

In [ ]:
#wähle deine favoriten aus
models = {
    "Gaussian": gs.Gaussian,
    #"Exponential": gs.Exponential,
    #"Matern": gs.Matern,
    #"Stable": gs.Stable,
    #"Rational": gs.Rational,
    #"Circular": gs.Circular,
    #"Spherical": gs.Spherical,
    #"SuperSpherical": gs.SuperSpherical,
    #"JBessel": gs.JBessel, #keine Varianzberechnung möglich !!
}

fig, ax = plt.subplots()
#betrachte nun das gefittete covarianz-modell
# plot the estimated variogram
ax.scatter(bin_center, gamma, color="k", label="data")


# fit all models to the estimated variogram
scores = {}
max_distance_variogram = 0.75*max(max_x-min_x, max_y-min_y)
for model in models:
    fit_model = models[model](dim=2)
    para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True, nugget=False)
    fit_model.plot(x_max=max_distance_variogram, ax=ax)
    scores[model] = r2

ranking = sorted(scores.items(), key=lambda item: item[1], reverse=True)
print("RANKING by Pseudo-r2 score")
best_model = {}
best_score = -1
for i, (model, score) in enumerate(ranking, 1):
    if score > best_score:
        best_score = score
        best_model = models[model](dim=2)
    print(f"{i:>6}. {model:>15}: {score:.5}")

# Aufgabe:
- Gehe zum vorherigen Punkt, reduziere die Anzahl der Stützpunkte (n_known_datapoints) und wiederhole die Schritte.
- Welche auswirkungen hat es auf variogram ?
- Ab welcher anzahl der Punkte würdest du keinen zuverlässigen Fit der Kovarianz-funktion erwarten?

# HANDS-ON: Varianzaufgabe 2 

In [ ]:
#Führe den gesamten Block aus
f_var_2 = f_shubert_1_not_for_hand_ons()
min_x, max_x, min_y, max_y = f_var_2.getRange()
x_p,y_p = getInitPoints("CenteredRandomGridCombinedPoints", 3000, min_x,max_x, min_y,max_y)
z_p = f_var_2.getValue(x_p,y_p)

fig, (ax1, ax2) = plt.subplots(1,2)

bins = np.linspace(0,3,40)
bin_center, gamma = gs.vario_estimate((x_p,y_p), z_p, bin_edges=bins)
bin_center = bin_center[gamma > 0]
gamma = gamma[gamma > 0]

models = {
    "JBessel": gs.JBessel,
}


# plot the estimated variogram
ax2.scatter(bin_center, gamma, color="k", label="data")

# fit all models to the estimated variogram
for model in models:
    fit_model = models[model](dim=2)
    para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True, nugget=False)
    fit_model.plot(x_max=5, ax=ax2)

plotFunction2D(f_var_2, ax1)
fig.set_size_inches(15,5)

# Aufgabe:
Im Plot links siehst du die wahre Funktion, links das entsprechende Variogram.
- Überlegt es euch, warum das Variogram so aussieht, wie es aussieht?
- Welche Bedeutung haben insbesondere Abstände zwischen den Schwingungen?

# HANDS-ON: Varianzaufgabe 3

In [ ]:
#Führe den gesamten Block aus
f_var_3 = f_shubert_2_not_for_hand_ons()
min_x, max_x, min_y, max_y = f_var_3.getRange()
x_p,y_p = getInitPoints("CenteredRandomGridCombinedPoints", 3000, min_x,max_x, min_y,max_y)
z_p = f_var_3.getValue(x_p,y_p)
fig, (ax1, ax2) = plt.subplots(1,2)

bins = np.linspace(0,3,40)
bin_center, gamma = gs.vario_estimate((x_p,y_p), z_p, bin_edges=bins)
bin_center = bin_center[gamma > 0]
gamma = gamma[gamma > 0]

models = {
    "JBessel": gs.JBessel,
}


# plot the estimated variogram
ax2.scatter(bin_center, gamma, color="k", label="data")

# fit all models to the estimated variogram
for model in models:
    fit_model = models[model](dim=2)
    para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True, nugget=True)
    fit_model.plot(x_max=5, ax=ax2)

plotFunction2D(f_var_3, ax1)

fig.set_size_inches(15,5)

# Aufgabe:
- Hat sich das Variogram geändert ?
- Falls ja, was könnte diese Änderung aussagen ?

# --------------------------------------------------------------------
# HANDS-ON: Kriging-Modelle erstellen und verstehen 
# --------------------------------------------------------------------

# HANDS-ON: Kriging 1, Manuelle Auswahl der Punkte

In [ ]:
#Erstelle deine eigene funktion, oder verwende eine der bereits definierten
class f_deine_eigene_funktion_2:
    def __init__(self):
        #definiere Wertebereich der Funktion
        self.min_x = 0
        self.max_x = 1
        self.min_y = 0
        self.max_y = 1

    def getRange(self):
        return self.min_x, self.max_x, self.min_y, self.max_y

    def getValue(self, x, y):
        #definiere funktion
        return x + y

#wähle die funktion aus
#f_bekannte_funktion_2 = f_deine_eigene_funktion_2()    #wenn du die von dir definierte funktion verwenden möchtest
#f_bekannte_funktion_2 = f_herbie_4()                         #wenn du z.b. die hier implementierte f_herbie_3
f_bekannte_funktion_2 = f_eggholder()                      #wenn du z.b. die hier implementierte f_eggholder() : <- durchaus interessant für diese aufgabe, da relativ komplex

#funktion zum anschauen der funktion
fig, ax = plt.subplots()
plotFunction2D(f_bekannte_funktion_2, ax)

In [ ]:
#extrahiere wertebereich
min_x, max_x, min_y, max_y = f_bekannte_funktion_2.getRange()



#lege startpunkte manuell / visuell fest
#entscheide dich für eine bestimmte anzahl "n_max_points" der startpunkte (tipp: nicht weniger als 15, je nach fkt-komplexität)

#wird im nächsten Abschnitt verwendet, bitte manuell diese Anzahl an Punkten auswählen 
n_max_points = 30 # <- Anpassen

x_p = []
y_p = []
#beispiel zum hinzufügen von vier eckpunkten punkten (x,y)
#für wertebereich 0-1
#x_p , y_p = addOnePoint(n_new, y_new, x_p , y_p)

# "n_max_points"mal die funktion verwenden
#x_p , y_p = addOnePoint(0.0, 0.0, x_p , y_p)
#x_p , y_p = addOnePoint(0.0, 1.0, x_p , y_p)
#x_p , y_p = addOnePoint(1.0, 0.0, x_p , y_p)
#x_p , y_p = addOnePoint(1.0, 1.0, x_p , y_p)
# ............
#x_p , y_p = addOnePoint(1.1, 2.0, x_p , y_p)

#berechne die funktionswerte für startpunkte
z_p = f_bekannte_funktion_2.getValue(x_p, y_p)

#fitte variogram manuell (aus der variogram-aufgabe copy-pasten) oder automatisch mit der folgenden funktion
best_model, best_score, bin_center, gamma = getBestCovarianceModel(x_p, y_p, z_p, True)

#falls ihr das variogram anschauen wollt, auskommentieren
#fig, ax1 = plt.subplots()
#plotCovarianceModel(bin_center, gamma, best_model, ax1)

#berechne kriging-model
krig = calculateKrigingModel("universal_quadratic", best_model, x_p, y_p, z_p)

#man kann das feld und sqrt(varianz) = std.abw anschauen
fig, (ax1, ax2) = plt.subplots(1,2)
plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
fig.set_size_inches(15,5)

#Berechnete quadratische Abweichung zur wahren Funktion
fig, ax1 = plt.subplots()
se_map, mse_manuell = getSEmapOfKrigModel(krig, f_bekannte_funktion_2, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1)

print("MSE manuelle Datenpunkteauswahl: %.5f"%mse_manuell)

# HANDS-ON: Kriging 2, Auswahl der Punkte mithilfe von max. erwarteten Modelfehler

In [ ]:
################# Die zu untersuchende Funktion bleibt aus manuellem Abschnitt         ##########

In [ ]:
#man muss mit bestimmten Anteil der Punkte starten (lass es mindestens 10 sein)
#die restlichen Punkte (gleiche Anzahl wie bei der manuellen Selektion) werden einzeln hinzugefügt

n_start_points = 20 #<- kleiner als "n_max_points" setzen

x_p,y_p = getInitPoints("CenteredRandomGridCombinedPointsWithEdges", n_start_points, min_x,max_x, min_y,max_y)
z_p = f_bekannte_funktion_2.getValue(x_p, y_p)
best_model, best_score, bin_center, gamma = getBestCovarianceModel(x_p, y_p, z_p, True)
krig = calculateKrigingModel("universal_quadratic", best_model, x_p, y_p, z_p)
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
plotCovarianceModel(bin_center, gamma, best_model, ax3)
fig.set_size_inches(25,5)


########## begin der zu wiederholende teil, bis alle Punkte hingefügt sind ##########
for i in range(n_max_points - n_start_points):
    #hinfüge den punkt mit dem größten Modelfehler (manuell/visuell oder mit funktion)
    x_new, y_new = getPointWithMaxValue(krig, min_x, max_x, min_y, max_y)
    z_new = f_bekannte_funktion_2.getValue(x_new, y_new)
    x_p = np.append(x_p, x_new)
    y_p = np.append(y_p, y_new)
    z_p = np.append(z_p, z_new)

    best_model, best_score, bin_center, gamma = getBestCovarianceModel(x_p, y_p, z_p, True)

    #berechne kriging-model
    krig = calculateKrigingModel("universal_quadratic", best_model, x_p, y_p, z_p)

    #man kann das feld und sqrt(varianz) = std.abw anschauen
    fig, (ax1, ax2, ax3) = plt.subplots(1,3)
    plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
    plotCovarianceModel(bin_center, gamma, best_model, ax3)
    ax1.scatter(x_new,y_new, color = "white", s=200)
    ax2.scatter(x_new,y_new, color = "white", s=200)
    fig.set_size_inches(25,5)
    
#im plot sind die neu hinzugefügten Punkte mit weiß gekennzeichnet

#Quadtrierte Abweichung von der wahren Funktion
fig, ax1 = plt.subplots()
se_map, mse_auto = getSEmapOfKrigModel(krig, f_bekannte_funktion_2, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1)
fig.set_size_inches(10,7)

print("MSE manuelle Datenpunkteauswahl: %.5f"%mse_manuell)
print("MSE Datenpunkteauswahl nach größtem Modelfehler: %.5f"%mse_auto)

if mse_manuell < mse_auto:
    print("\nGlückwusch !! Ihr wart bei der Selektion der Stützpunkte besser, als automatisches Hinfügen nach großtem Modelfehler !!\n")



# Aufgabe:
- Wird mit jedem hinzugefügten Punkt das Variogram besser ?
- Wird mit jedem hinzugefügten Punkt die vorhersagte Funktion besser ?
- Habt ihr die wichtigen Stützpunkte besser geschätzt ? (Ist der Fehler im ersten Fall kleiner)

# HANDS-ON: Eine unbekannte Funktion mit minimaler Anzahl der Stützpunkten nähern 

In [ ]:
#wähle die funktion aus (eine, die ihr noch nicht angeschaut habt)
#f_unbekannte_funktion = f1()                              # kann man nehmen, muss man nicht
#f_unbekannte_funktion = f_branin()                        # kann man nehmen, ist OK <- eher einfache funktion
#f_unbekannte_funktion = f_six_hump_camel()                # ist i.O. für diese aufgabe
f_unbekannte_funktion = f_eggholder()                      # durchaus interessant für diese aufgabe, da relativ komplex, falls in der vorherigen aufgabe nicht genommen

#extrahiere wertebereich
min_x, max_x, min_y, max_y = f_unbekannte_funktion.getRange()

n_start_points = 15
x_p,y_p = getInitPoints("CenteredRandomGridCombinedPointsWithEdges", n_start_points, min_x,max_x, min_y,max_y)

#berechne die funktionswerte für startpunkte
z_p = f_unbekannte_funktion.getValue(x_p, y_p)

#fitte variogram manuell (aus der variogram-aufgabe copy-pasten) oder automatisch mit der folgenden funktion
best_model, best_score, bin_center, gamma = getBestCovarianceModel(x_p, y_p, z_p, True)
#falls ihr das variogram anschauen wollt, auskommentieren
#fig, ax1 = plt.subplots()
#plotCovarianceModel(bin_center, gamma, best_model, ax1)

#berechne kriging-model
krig = calculateKrigingModel("universal_quadratic", best_model, x_p, y_p, z_p)

#man kann das feld, sqrt(varianz) = std.abw  und variogram anschauen
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
plotCovarianceModel(bin_center, gamma, best_model, ax3)
fig.set_size_inches(20,5)


In [ ]:
#das ist die Ausgangszustand. Versucht mit möglichts wenigen zusätzlichen Punkten bestmögliche Beschreibung der Funktion erhalten 
#(die ihr eigentlich nicht kennt)

###### diesen schritt mehrfach ausführen (bzw. copy-pasten und in eine neue code-zeile einfügen, um vorherigen zustand zu speichern), bis ihr mitm ergebniss zufrieden seid

# hinfüge den punkt mit dem größten Modelfehler oder den manuellen punkt, der ihr glaubt würde den fit verbessern

#hier mit dem größten SE
x_new, y_new = getPointWithMaxValue(krig, min_x, max_x, min_y, max_y) #oder manuell festlegen
x_p , y_p = addOnePoint(x_new, y_new, x_p , y_p)
#hier manuell (auskommentieren, die obere funktion kommentieren)
#x_p , y_p = addOnePoint(0.0, 0.0, x_p , y_p)

z_new = f_unbekannte_funktion.getValue(x_new, y_new)
z_p = np.append(z_p, z_new)

best_model, best_score, bin_center, gamma = getBestCovarianceModel(x_p, y_p, z_p, True)

# berechne kriging-model
krig = calculateKrigingModel("universal_quadratic", best_model, x_p, y_p, z_p)

# man kann das feld und sqrt(varianz) = std.abw anschauen
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
plotCovarianceModel(bin_center, gamma, best_model, ax3)
ax1.scatter(x_new,y_new, color = "white", s=200)
ax2.scatter(x_new,y_new, color = "white", s=200)
fig.set_size_inches(20, 5)

print("Anzahl der Punkte im Model: %d"%len(z_p))

# Aufgabe:
- Mit wie vielen Punkten konntet ihr die "wahre" Funktion erahnen?
- Diese Teilaufgabe kann mit anderen Funktionen, wenn ihr Lust habt, wiederholt werden
- Diese Teilaufgabe kann mit der anderen Krigingmethode ("simple","ordinary","universal_linear","universal_quadratic") wiederholt werden

# HANDS-ON: Signifikanz der Modelgenauigkeit vs. Datenpunkteanzahl

Euch steht die Funktion analyseFunction(f1, n_points_to_fit, n_runs, init_fraction, fig, ax1, ax2, ax3, ax4) zur Verfügung,
Mit dieser Funktion kann der MSE in Abhängigkeit von der Anzahl der Startpunkte untersuchen werden.
Man kann in diesem notebook definierte Funktion damit untersuchen
Die Logik führt zum Einen den Fit mit allen Punkten durch, zum Anderen wird zusätzlich mit Aufsplittung in Startpunkte + größter SE das Model laufend verbessert


In [ ]:
#wähle die Funktion zur statistischen Analyse aus
f_funktion_zum_analysieren = f_herbie_3()

n_points_to_fit = [15, 20, 25, 30] #anzahl der stützpunkte
n_runs = 10 #anzahl der wiederholungen mit gleichen parametern (übertreibt nicht)
init_faction = 0.7 #anteil an startpunkten zum ersten berechnen des models, der rest wird nach dem größten SE hinzugefügt
krig_type = "universal_quadratic" #("simple","ordinary","universal_linear","universal_quadratic")
start_points = "CenteredRandomGridCombinedPointsWithEdges" #oben nachschauen

#Laufzeit mit den nicht veränderten Parametern ca. 5 min
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4)
model_mse, model_mse_std, best_krige, best_cov_model, best_bin_center, best_gamma = analyseFunction(f_funktion_zum_analysieren,n_points_to_fit, n_runs, init_faction, fig, ax1, ax2, ax3, ax4, krig_type, start_points)

#die Funktion "analyseFunction" plottet das best gefundene Modell (wird wohl das, mit den meisten Punkten sein)


# Aufgabe:
- War das Ergebnis zu erwarten? 
- Diese Teilaufgabe kann mit anderen Funktionen, wenn ihr Lust habt, wiederholt werden
- Diese Teilaufgabe kann mit anderen init_fraction - Werten, wenn ihr Lust habt, wiederholt werden
- Diese Teilaufgabe kann mit der anderen Krigingmethode ("simple","ordinary","universal_linear","universal_quadratic") wiederholt werden
- Ihr könnt für die von euch manuell/visuell geschätzte Funktion hier durchlaufen lassen um "euren" Modelfehler besser einordnen zu können 